In [ ]:
import zipfile
import os
from deepface import DeepFace
import matplotlib.pyplot as plt
import cv2
import pandas as pd

# Step 1: Automatically use captured_frames.zip in the same directory
zip_filename = "captured_frames.zip"

# Check if the file exists
if os.path.exists(zip_filename):
    # Extract zip file
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall("images")

    # Verify extraction
    if not os.path.exists("images"):
        print("Image extraction failed.")
    else:
        print(f"Images extracted successfully to: {os.path.abspath('images')}")
else:
    print(f"File '{zip_filename}' not found. Please ensure it is in the same directory.")

# Step 2: Define function to analyze emotions with multiple backends
def analyze_emotions(image_path, backend):
    print(f"Analyzing {image_path} with {backend}...")
    results = {}
    
    try:
        print(f"Using backend: {backend}")
        result = DeepFace.analyze(image_path, actions=['emotion'], enforce_detection=False, detector_backend=backend)  # Specify backend
        
        # Print the structure of the result for debugging
        print(f"Result structure for {backend}: {result}")

        emotion_scores = result[0]['emotion']  # Get emotion scores for all emotions
        dominant_emotion = max(emotion_scores, key=emotion_scores.get)  # Get the dominant emotion
        emotion_score = emotion_scores[dominant_emotion]  # Get the score for the dominant emotion
        results[backend] = (dominant_emotion, emotion_score, emotion_scores)  # Store emotion and scores
        print(f"Backend: {backend}, Detected emotion: {dominant_emotion}, Score: {emotion_score}")
        print(f"Full emotion scores: {emotion_scores}")
    except Exception as e:
        print(f"Error analyzing {image_path}: {e}")
    
    return results

# Step 3: Process each image in the folder (and subfolders) and display the result
backends = ['opencv', 
            'mtcnn',
            'retinaface',]  # List of backends to use
results = {}

# Use os.walk to find images in all subdirectories
images = []
for root, dirs, files in os.walk("images"):
    for filename in files:
        if filename.endswith(".jpg") or filename.endswith(".png"):
            filepath = os.path.join(root, filename)
            images.append(filepath)

# Loop through each backend
for backend in backends:
    print(f"Starting analysis with {backend}")
    for image_path in images:
        emotion_results = analyze_emotions(image_path, backend)

        # Store results in a dictionary
        filename = os.path.basename(image_path)
        if filename not in results:
            results[filename] = {}
        results[filename][backend] = emotion_results[backend]

        # Display result
        img = cv2.imread(image_path)
        if img is not None:
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            emotion = emotion_results[backend][0]  # Get dominant emotion from the current backend
            plt.title(f"Emotion: {emotion}")
            plt.axis("off")
            plt.show()
        else:
            print(f"Could not read image: {filename}")

# Optional: Display all results at the end
print("Emotion analysis results:")
for filename, emotion_data in results.items():
    print(f"{filename}:")
    for backend_name, (dominant_emotion, emotion_score, emotion_scores) in emotion_data.items():
        print(f"  Backend: {backend_name}, Dominant Emotion: {dominant_emotion}, Score: {emotion_score}")
        
# Save results to CSV
emotion_list = []
for filename, emotion_data in results.items():
    for backend_name, (dominant_emotion, emotion_score, emotion_scores) in emotion_data.items():
        emotion_list.append([filename, backend_name, dominant_emotion, emotion_score, emotion_scores])
        
# Create DataFrame and save to CSV
df = pd.DataFrame(emotion_list, columns=["Filename", "Backend", "Dominant Emotion", "Score", "Full Emotion Scores"])
df.to_csv("backend_results.csv", index=False)
